In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df_get_back=pd.read_table(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\1A39會員位置(無正確地址的資料)_一般卡_2020LOSS.txt',sep=',')
print(df_get_back.shape)

(3781, 4)


In [4]:
result=pd.read_table(r'C:\Users\RT009223\TestriteGroup\滲透率專案\02_adjusted data\1A39會員位置(經整理_配對村里)_一般卡_2020LOSS.txt',sep=',')
print(result.shape)

(28903, 7)


In [5]:
df_get_back=df_get_back[-df_get_back.id.isin(result.id)]
print(df_get_back.shape)
result=pd.concat([result,df_get_back]).reset_index(drop=True)
print(result.shape)

(3781, 4)
(32684, 7)


In [6]:
result.head(5)

,id,addr,lon,lat,縣市,行政區,村里
0,20090719000525,彰化縣彰化市下?里28鄰茄苳路一段129號 ...,120.545339,24.092241,彰化縣,彰化市,下廍里
1,20060312000702,高雄市美濃區龍山里4鄰文化街10號 ...,120.573339,22.882576,高雄市,美濃區,龍山里
2,20100341150738,臺中市南區永和里11鄰高工路510號 ...,120.669786,24.114502,台中市,南區,永和里
3,20110401000338,臺中市北區淡溝里30鄰臺灣大道二段362號 ...,120.665160,24.154520,台中市,北區,淡溝里
4,20120624000723,彰化縣溪湖鎮西溪里15鄰員鹿路二段459巷24號 ...,120.484457,23.956234,彰化縣,溪湖鎮,西溪里


In [12]:
sales=pd.read_csv(r'C:\Users\RT009223\TestriteGroup\滲透率專案\01_raw data\1A39_2020LOSS一般卡會員消費金額.csv',error_bad_lines=False,header=None)
sales.columns=['id','site','tick','sale','trans']
print(sales.shape)
sales.head(5)

(34611, 5)


,id,site,tick,sale,trans
0,19960203001223,NaN,NaN,NaN,NaN
1,19960309000120,NaN,NaN,NaN,NaN
2,19960317000097,NaN,NaN,NaN,NaN
3,19960327000027,NaN,NaN,NaN,NaN
4,19960915000269,NaN,NaN,NaN,NaN


In [13]:
sales=sales.groupby(['id','site'])['sale'].sum().reset_index()
sales.head(5)

,id,site,sale
0,18991230000206,1A36,11599.28
1,18991230000312,1A36,1090.00
2,19960101002146,1A11,2861.00
3,19960126000104,1G46,9894.28
4,19960128000171,1A12,54.00


In [14]:
result2=pd.merge(result,sales,how='outer')

In [16]:
print(result2.shape)
result2.head(5)

(34611, 9)


,id,addr,lon,lat,縣市,行政區,村里,site,sale
0,20090719000525,彰化縣彰化市下?里28鄰茄苳路一段129號 ...,120.545339,24.092241,彰化縣,彰化市,下廍里,1A10,7271.7
1,20060312000702,高雄市美濃區龍山里4鄰文化街10號 ...,120.573339,22.882576,高雄市,美濃區,龍山里,1A14,228.0
2,20100341150738,臺中市南區永和里11鄰高工路510號 ...,120.669786,24.114502,台中市,南區,永和里,NaN,NaN
3,20110401000338,臺中市北區淡溝里30鄰臺灣大道二段362號 ...,120.665160,24.154520,台中市,北區,淡溝里,NaN,NaN
4,20120624000723,彰化縣溪湖鎮西溪里15鄰員鹿路二段459巷24號 ...,120.484457,23.956234,彰化縣,溪湖鎮,西溪里,1A12,1285.0


In [17]:
result2.loc[(result2['sale'].isna())|(result2['sale']<0),['sale']]=0

In [18]:
result2_2=result2.groupby(['id'])['sale'].sum().reset_index(name='sum')

In [19]:
result3=pd.merge(result2,result2_2)

In [20]:
result3['share']=result3['sale']/result3['sum']

In [21]:
result3.loc[result3['share'].isna(),['share']]=1

In [22]:
result3.share.sum()

32684.0

In [23]:
result3.loc[result3['site'].isna(),['site']]='NA'

In [24]:
result4=result3.groupby(['site'])['share'].sum().reset_index(name='n').sort_values(by='n',ascending=False).reset_index(drop=True)

In [25]:
result4['share']=result4.n/result4.n.sum()

In [26]:
result4.head(50)

,site,n,share
0,NA,23486.000000,0.718578
1,1A11,3384.934610,0.103565
2,1A36,1884.835665,0.057668
3,1A10,453.018627,0.013861
4,1A34,400.738149,0.012261
5,1A12,311.546473,0.009532
6,1A07,251.272722,0.007688
7,1G46,231.364287,0.007079
8,1A32,230.707698,0.007059
9,1A05,224.269533,0.006862
